# Challenge LATAM

## Preámbulo
Primero, solo con ver el archivo json en un editor de texto, ya se pueden notar algunas cosas:
1. La documentación de las instrucciones no se corresponde con la versión de la API de los datos.
2. Cada linea del archivo corresponde a un entrada/objeto distinto, los cuales son tweets.

Luego, para poder visualizar y entender mejor la estructura de cada objeto json utilizo el comando de shell:

```Bash
cat farmers-protest-tweets-2021-2-4.json | jq
```

el cual realiza un "pretty print", es decir, imprime el json parseado y coloreado en la terminal.

## Respecto a los Datos

Se puede notar que el json tiene un peso considerable ($≈380$ mb) considerando que es solo un archivo de texto, por lo que podría ser una buena idea evitar tener el total de los datos en memoria a la hora de procesarlos.

Lo anterior, sumado a que por su tamaño tampoco se puede subir a github (limite de 100mb por archivo), me hizo decidir dividir el archivo en partes.

La manera más elegante de hacerlo sería leer el json como stream e ir manejando un buffer de tamaño menor a $100$ mb, que al llenarse, escriba su contenido a un archivo json y luego se vacíe.

Sin embargo en este caso es mucho más fácil dividir el archivo por lineas, en partes lo más iguales posibles. Para esto, cree un script de shell ('../utils/file_split.sh') el cual divide el json en $N$ archivos de $\approx L/N$ lineas c/u donde $L$ es el número de lineas del archivo y $N$ un argumento, el cual decidí que fuera 5.

Finalmente se obtuvo 5 archivos de $\approx 77$ mb, y se pudieron subir a github.

## Soluciones

Por temas de comodidad y orden, estructuré las soluciones de las 3 pruebas como módulos, cada uno en su propio directorio dentro de './src'. Además, los datos se ubicaron en la carpeta './data'.

En cada archivo '\__init__.py' de los módulos cree una "interfaz" o intermediario para la ejecución de las funciones. Esto con el fin de ejecutarlas en paralelo, aprovechando la partición previa de los datos, y optimizando el tiempo de ejecución para todas las funciones.

_Cada solución está más detalla en sus códigos correspondientes._

### Imports

In [1]:
import os
import pandas as pd
from q1 import q1_time, q1_memory
from q2 import q2_time, q2_memory
from q3 import q3_time, q3_memory

data_dir = "../data"
json_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.json')]

%load_ext memory_profiler

### Problema 1

La idea principal para la resolución de ambas versiones fue, comparar el rendimiento de el diccionario tradicional usado como contador, y otra versión donde además se agrega alguna otra estructura de datos que se ajuste bien al tipo de consultas que se hacen. En este caso, se utiliza el heap, con la idea de poder acceder a los elementos máximos con mayor rapidez.

In [14]:
q1t = q1_time(json_files)
q1m = q1_memory(json_files)

#### Profiling

In [25]:
%timeit q1_time(json_files);
%timeit q1_memory(json_files);

2.52 s ± 34.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.37 s ± 8.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
%memit q1_time(json_files);
%memit q1_memory(json_files);

peak memory: 159.59 MiB, increment: 0.50 MiB
peak memory: 156.17 MiB, increment: 0.16 MiB


Para el tiempo, no se cumple lo predicho a través del análisis teórico. Esto se puede deber a varias cosas, entre ellas, que al paralelizar y crear más ejecuciones, la construcción de los heaps produjo tiempo adicional. Esto se podría demostrar realizando una única ejecución con el archivo json completo. 

La otra opción sería que, heapq, el módulo para utilizar heaps en python, sea más una "emulación" de la estructura, ya que necesita que se le entregue una lista de python para funcionar, lo que puede provocar que no se refleje el aporte de esta estructura debido a su construcción a tan alto nivel.

Finalmente, esta la opción que en la práctica, simplemente ocurren muchos procesos por debajo que hacen el usar un diccionario más eficiente.

#### Resultados

In [15]:
df1 = pd.DataFrame(q1t, columns=['fecha', 'usuario'])
print("Top 10 fechas con más tweets junto con el usuario que más tweets hizo en esa fecha")
df1

Top 10 fechas con más tweets junto con el usuario que más tweets hizo en esa fecha


,fecha,usuario
0,2021-02-12,RanbirS00614606
1,2021-02-13,rebelpacifist
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,rebelpacifist
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


In [16]:
df2 = pd.DataFrame(q1m, columns=['fecha', 'usuario'])
print("Top 10 fechas con más tweets junto con el usuario que más tweets hizo en esa fecha")
df2

Top 10 fechas con más tweets junto con el usuario que más tweets hizo en esa fecha


,fecha,usuario
0,2021-02-12,RanbirS00614606
1,2021-02-13,rebelpacifist
2,2021-02-17,RaaJVinderkaur
3,2021-02-16,jot__b
4,2021-02-14,rebelpacifist
5,2021-02-18,rebelpacifist
6,2021-02-15,jot__b
7,2021-02-20,MangalJ23056160
8,2021-02-23,Surrypuria
9,2021-02-19,Preetm91


_Considerar: Al utilizar distintos métodos para las soluciones, se puede afectar la consistencia de los resultados cuando hay empates en la frecuencia._

### Problema 2

In [2]:
q2t = q2_time(json_files)
q2m = q2_memory(json_files)

#### Profiling

In [3]:
%timeit q2_time(json_files);
%timeit q2_memory(json_files);

9.17 s ± 93.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
9.52 s ± 79.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%memit q2_time(json_files);
%memit q2_memory(json_files);

peak memory: 178.09 MiB, increment: 12.03 MiB
peak memory: 171.72 MiB, increment: 0.16 MiB


La serialización de los datos a dataframe, y la ejecución de funciones a través del método ".apply()" no convierten la obtención de resultados en un proceso más lento que ir leyendo linea por linea e ir procesando los textos inmediatamente.

#### Results

In [5]:
print("Top 10 emojis más utilizados")
df3 = pd.DataFrame(q2t, columns=['emoji', 'frecuencia'])
df3

Top 10 emojis más utilizados


,emoji,frecuencia
0,🙏,5049
1,😂,3072
2,🚜,2972
3,🌾,2182
4,🇮🇳,2086
5,🤣,1668
6,✊,1651
7,❤️,1382
8,🙏🏻,1317
9,💚,1040


In [6]:
print("Top 10 emojis más utilizados")
df4 = pd.DataFrame(q2m, columns=['emoji', 'frecuencia'])
df4

Top 10 emojis más utilizados


,emoji,frecuencia
0,🙏,5049
1,😂,3072
2,🚜,2972
3,🌾,2182
4,🇮🇳,2086
5,🤣,1668
6,✊,1651
7,❤️,1382
8,🙏🏻,1317
9,💚,1040


### Problema 3
La similitud del problema con una red me dio la idea de crear una versión de la solución como grafo, donde la solución está en extraer los 10 nodos con mayor grado de entrada, y la otra solución, simplemente contar la frecuencia con un diccionario.

In [21]:
q3t = q3_time(json_files)
q3m = q3_memory(json_files)

#### Profiling

In [30]:
%timeit q3_time(json_files);
%timeit q3_memory(json_files);

2.45 s ± 21.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.32 s ± 9.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%memit q3_time(json_files);
%memit q3_memory(json_files);

peak memory: 161.94 MiB, increment: 0.20 MiB
peak memory: 161.95 MiB, increment: 0.16 MiB


El que los resultados sean tan similares es interesante por el hecho que tener los datos almacenados como grafos, da mayores ventajas en cuanto a la extracción de información valiosa, partiendo por que se podrían utilizar otras medidas de centralidad o "importancia" como PageRank, betwenness, entre otros.

#### Results

In [22]:
df5 = pd.DataFrame(q3t, columns=['usuario', 'frecuencia'])
print("Top 10 usuarios con más menciones")
df5

Top 10 usuarios con más menciones


,usuario,frecuencia
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926


In [23]:
df6 = pd.DataFrame(q3m, columns=['usuario', 'frecuencia'])
print("Top 10 usuarios con más menciones")
df6

Top 10 usuarios con más menciones


,usuario,frecuencia
0,narendramodi,2265
1,Kisanektamorcha,1840
2,RakeshTikaitBKU,1644
3,PMOIndia,1427
4,RahulGandhi,1146
5,GretaThunberg,1048
6,RaviSinghKA,1019
7,rihanna,986
8,UNHumanRights,962
9,meenaharris,926
